# Introduction

In this document, we will be precribing the template to solve the Kuramoto-Sivashinsky (K-S) equation. In this instance, the stencil in space will be fourth (4th) order, and for the time integration, we will be using the fourth (4th) order Runge-Kutta method.

The K-S equation is:

$$

\frac{\partial u}{\partial t} + u\frac{\partial u}{\partial x} + \alpha\frac{\partial^2 u}{\partial x^2} + \beta\frac{\partial^3 u}{\partial x^3} + \gamma\frac{\partial^4 u}{\partial x^4} = 0

$$


In [1]:
import sys
import os
import time
import numpy as np
import matplotlib.pyplot as plt

# Add the directory containing your module to sys.path
module_path = os.path.abspath(os.path.join('..', r"A:\Users\mtthl\Documents\Education\ME5653_CFD\git\me5653_CFD_repo\code\lib" ))
sys.path.append(module_path)

from distributedObjects import *
from distributedFunctions import *


# Spatial Stencil

The spatial stencil to be 4th order, and thus take four (4) points plus the sampling point, thus as below for interior points.

-X-------X-------X--------X-------X-

i-2______i-1______i_______i+1_____i+2

For the boundary points, the stencil will be:

X-------X-------X-------X-------X-

i_______i+1____i+2_____i+3_____i+4

-X-------X-------X-------X-------X

i-4______i-3_____i-2______i-1______i

Thus, the formulation for the spatial stencil in the 1st derivative becomes


In [2]:
first_interior_gradient = numericalGradient( 1 , ( 3 , 3 ) )
first_interior_gradient.coeffs


array([-1.66666667e-02,  1.50000000e-01, -7.50000000e-01,  3.70074342e-16,
        7.50000000e-01, -1.50000000e-01,  1.66666667e-02])

In [3]:
first_LHS_gradient = numericalGradient( 1 , ( 0 , 6 ) )
first_LHS_gradient.coeffs


array([-2.45      ,  6.        , -7.5       ,  6.66666667, -3.75      ,
        1.2       , -0.16666667])

In [4]:
first_RHS_gradient = numericalGradient( 1 , ( 6 , 0 ) )
first_RHS_gradient.coeffs


array([ 0.16666667, -1.2       ,  3.75      , -6.66666667,  7.5       ,
       -6.        ,  2.45      ])

<u>Interior 1st Derivative - 6th Order<u>

$$

\frac{\partial \phi}{\partial x}=\frac{ -\frac{1}{60}\phi_{i-3} + \frac{3}{20}\phi_{i-2} - \frac{3}{4}\phi_{i-1} + \frac{3}{4}\phi_{i+1} - \frac{3}{20}\phi_{i+2} + \frac{1}{60}\phi_{i+3} }{\Delta x}

$$

<u>Boundary LHS 1st Derivative - 6th Order<u>

$$

\frac{\partial \phi}{\partial x}=\frac{ -\frac{49}{20}\phi_{i} + 6\phi_{i+1} - \frac{15}{2}\phi_{i+2} + \frac{20}{3}\phi_{i+3} - \frac{15}{4}\phi_{i+4} + \frac{6}{5}\phi_{i+5} - \frac{1}{6}\phi_{i+6}  }{\Delta x}

$$

<u>Boundary RHS 1st Derivative - 6th Order<u>

$$

\frac{\partial \phi}{\partial x}=\frac{ \frac{1}{6}\phi_{i-6} -\frac{6}{5}\phi_{i-5} + \frac{15}{4}\phi_{i-4} - \frac{20}{3}\phi_{i-3} + \frac{15}{2}\phi_{i-2} - 6\phi_{i-1} + \frac{49}{20}\phi_{i} }{\Delta x}

$$



The 2nd derivative becomes


In [5]:
second_interior_gradient = numericalGradient( 2 , ( 3 , 3 ) )
second_interior_gradient.coeffs


array([ 0.01111111, -0.15      ,  1.5       , -2.72222222,  1.5       ,
       -0.15      ,  0.01111111])

In [6]:
second_LHS_gradient = numericalGradient( 2 , ( 0 , 6 ) )
second_LHS_gradient.coeffs


array([  4.51111111, -17.4       ,  29.25      , -28.22222222,
        16.5       ,  -5.4       ,   0.76111111])

In [7]:
second_RHS_gradient = numericalGradient( 2 , ( 6 , 0 ) )
second_RHS_gradient.coeffs


array([  0.76111111,  -5.4       ,  16.5       , -28.22222222,
        29.25      , -17.4       ,   4.51111111])

<u>Interior 2nd Derivative - 6th Order<u>

$$

\frac{\partial^2 \phi}{\partial x^2}=\frac{ \frac{1}{90}\phi_{i-3} - \frac{3}{20}\phi_{i-2} + \frac{3}{2}\phi_{i-1} - \frac{245}{90}\phi_{i} + \frac{3}{2}\phi_{i+1} - \frac{3}{20}\phi_{i+2} + \frac{1}{90}\phi_{i+3} }{\Delta x^2}

$$

<u>Boundary LHS 2nd Derivative - 6th Order<u>

$$

\frac{\partial^2 \phi}{\partial x^2}=\frac{ \frac{406}{90}\phi_{i} - \frac{87}{5}\phi_{i+1} + \frac{117}{4}\phi_{i+2} - \frac{254}{9}\phi_{i+3} + \frac{33}{2}\phi_{i+4} - \frac{27}{5}\phi_{i+5} + \frac{137}{180}\phi_{i+6} }{\Delta x^2}

$$

<u>Boundary RHS 2nd Derivative - 6th Order<u>

$$

\frac{\partial^2 \phi}{\partial x^2}=\frac{ \frac{137}{180}\phi_{i-6} - \frac{27}{5}\phi_{i-5} + \frac{33}{2}\phi_{i-4} - \frac{254}{9}\phi_{i-3} + \frac{117}{4}\phi_{i-2} - \frac{87}{5}\phi_{i-1} + \frac{406}{90}\phi_{i} }{\Delta x^2}

$$



Even though we will not be using the 3rd derivative, here it is


In [8]:
third_interior_gradient = numericalGradient( 3 , ( 3 , 3 ) )
third_interior_gradient.coeffs


array([ 0.125, -1.   ,  1.625,  0.   , -1.625,  1.   , -0.125])

In [9]:
third_LHS_gradient = numericalGradient( 3 , ( 0 , 6 ) )
third_LHS_gradient.coeffs


array([ -6.125,  29.   , -57.625,  62.   , -38.375,  13.   ,  -1.875])

In [10]:
third_RHS_gradient = numericalGradient( 3 , ( 6 , 0 ) )
third_RHS_gradient.coeffs


array([  1.875, -13.   ,  38.375, -62.   ,  57.625, -29.   ,   6.125])

<u>Interior 3rd Derivative - 6th Order<u>

$$

\frac{\partial^3 \phi}{\partial x^3}=\frac{ \frac{1}{8}\phi_{i-3} - \phi_{i-2} + \frac{13}{8}\phi_{i-1} - \frac{13}{8}\phi_{i+1} + \phi_{i+2} - \frac{1}{8}\phi_{i+3} }{\Delta x^3}

$$

<u>Boundary LHS 3rd Derivative - 6th Order<u>

$$

\frac{\partial^3 \phi}{\partial x^3}=\frac{ -\frac{49}{8}\phi_{i} + 29\phi_{i+1} - \frac{461}{8}\phi_{i+2} + 62\phi_{i+3} - \frac{307}{8}\phi_{i+4} + 13\phi_{i+5} - \frac{15}{8}\phi_{i+6}  }{\Delta x^3}

$$

<u>Boundary RHS 3rd Derivative - 6th Order<u>

$$

\frac{\partial^3 \phi}{\partial x^3}=\frac{ \frac{15}{8}\phi_{i-6} - 13\phi_{i-5} + \frac{307}{8}\phi_{i-4} - 62\phi_{i-3} + \frac{461}{8}\phi_{i-2} - 29\phi_{i-1} + \frac{49}{8}\phi_{i}  }{\Delta x^3}

$$


Finally, the fourth (4th) derivative will be


In [11]:
fourth_interior_gradient = numericalGradient( 4 , ( 3 , 3 ) )
fourth_interior_gradient.coeffs


array([-0.16666667,  2.        , -6.5       ,  9.33333333, -6.5       ,
        2.        , -0.16666667])

In [12]:
fourth_LHS_gradient = numericalGradient( 4 , ( 0 , 6 ) )
fourth_LHS_gradient.coeffs


array([  5.83333333, -31.        ,  68.5       , -80.66666667,
        53.5       , -19.        ,   2.83333333])

In [13]:
third_RHS_gradient = numericalGradient( 4 , ( 6 , 0 ) )
third_RHS_gradient.coeffs


array([  2.83333333, -19.        ,  53.5       , -80.66666667,
        68.5       , -31.        ,   5.83333333])

<u>Interior 4th Derivative - 6th Order<u>

$$

\frac{\partial^4 \phi}{\partial x^4}=\frac{ -\frac{1}{6}\phi_{i-3} + 2\phi_{i-2} - \frac{13}{2}\phi_{i-1} + \frac{28}{3}\phi_{i} - \frac{13}{2}\phi_{i+1} + 2\phi_{i+2} - \frac{1}{6}\phi_{i+3} }{\Delta x^4}

$$

<u>Boundary LHS 4th Derivative - 6th Order<u>

$$

\frac{\partial^4 \phi}{\partial x^4}=\frac{ \frac{105}{18}\phi_{i} - 31\phi_{i+1} + \frac{137}{2}\phi_{i+2} - \frac{242}{3}\phi_{i+3} + \frac{107}{2}\phi_{i+4} - 19\phi_{i+5} + \frac{17}{6}\phi_{i+6} }{\Delta x^4}

$$

<u>Boundary RHS 4th Derivative - 6th Order<u>

$$

\frac{\partial^4 \phi}{\partial x^4}=\frac{ \frac{17}{6}\phi_{i-6} - 19\phi_{i-5} + \frac{107}{2}\phi_{i-4} - \frac{242}{3}\phi_{i-3} + \frac{137}{2}\phi_{i-2} - 31\phi_{i-1} + \frac{105}{18}\phi_{i} }{\Delta x^4}

$$


# Time Integration Method

As mentioned before, we will be using the Runge-Kutta 4th order time integration scheme (RK4). I chose this one because it seems to be more widely used, and thus would like to get some experience with it.

As formulated in [1], the RK4 method comes down to a central equation as follows:

$$

\phi^{n+1}=\phi^{n}+\frac{\Delta t}{6}\left( R^{n}+2R^{(1)}+2R^{(2)}+R^{(3)} \right)

$$

where $R$ is the time derivative linear operator that is a function of $u$ and $\phi$. All values in the parenthesis in superscript represent a virtual step between the time steps. The process to find the $R$-values is:

1. $ \phi^{(1)}=\phi^{n}+\frac{\Delta t}{2}R^{n} $

2. $ \phi^{(2)}=\phi^{n}+\frac{\Delta t}{2}R^{(1)} $

3. $ \phi^{(3)}=\phi^{n}+\Delta t R^{(2)} $

The $R^{(3)}$ value comes from the time derivative value that corresponds to $\phi^{(3)}$.



# Boundary Condition Formulation

As one can see in the K-S equation, there is a 4th derivative of $u$, which means that there needs to be four (4x) boundary conditions (BCs) that constrain the values in the function.

I am projecting what the future will be, but since the test functions will likely be trigonometric, the function will be periodic in the domain, or periodic-like. Thus, at the two ends of the domain, the value for $u$ will be held constant. Thus, the time integration scheme will be bypassed resulting in the following formulation:

$$

\begin{bmatrix} 1 \end{bmatrix} \begin{bmatrix} u \end{bmatrix}^{n+1}=\begin{bmatrix} 1 \end{bmatrix} \begin{bmatrix} u \end{bmatrix}^{n}

$$

There is a helpful property of trigonometric functions in that every other derivative results in the negative of a derivative. Thus, the second (2nd) derivative is the negative of the function, and the fourth (4th) derivative is the negative of the second (2nd) derivative, or the original function. Thus:

$$

\begin{bmatrix} 1 \end{bmatrix} \begin{bmatrix} \frac{\partial^2 u}{\partial x^2} \end{bmatrix}^{n+1}=\begin{bmatrix} -1 \end{bmatrix} \begin{bmatrix} u \end{bmatrix}^{n}

$$

or

$$

\begin{bmatrix} 1 \end{bmatrix} \begin{bmatrix} \frac{\partial^4 u}{\partial x^4} \end{bmatrix}^{n+1}=\begin{bmatrix} 1 \end{bmatrix} \begin{bmatrix} u \end{bmatrix}^{n}

$$

or

$$

\begin{bmatrix} 1 \end{bmatrix} \begin{bmatrix} \frac{\partial^4 u}{\partial x^4} \end{bmatrix}^{n+1}=\begin{bmatrix} -1 \end{bmatrix} \begin{bmatrix} \frac{\partial^2 u}{\partial x^2} \end{bmatrix}^{n}

$$

Now, there is no guarantee that the function will be a trigonometric one. However, it is known that this is a chaotic equation [2], and thus, small changes in conditions are important. It ultimately depends on the use of the equation, but the KS equation is typically used to describe physical phenomena, and thus we may be more interested in the flux behavior at the boundaries. When the values at the boundaries are held the same, this results in a zero-flux condition. Thus,

$$

\begin{bmatrix} 1 \end{bmatrix} \begin{bmatrix} v \end{bmatrix} = 0 ,where\: v=\frac{u^2}{2}

$$

Additionally, one may assume that the boundaries have no diffusivity. Thus,

$$

\begin{bmatrix} 1 \end{bmatrix} \begin{bmatrix} \frac{\partial^2 u}{\partial x^2} \end{bmatrix}^{n}=0

$$

Again, it will depend on what we are modeling with our solution.


# Works Cited

1. Anderson, D. A., Tannehill, J. C., Pletcher, R. H., Munipalli, R., and Shankar, V. (2021). $ \underline{\text{Computational Fluid Mechanics and Heat Transfer}} $. 4th Edition. CRC Press.

2. Boghosian, B. M., Chow, C. C., and Hwa, T. (1999). $ \underline{\text{Hydrodynamics of the Kuramoto-Sivashinsky Equation in Two Dimensions}} $. Physical Review Letters. Vol 83, No 25.

